In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Launch browser
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.google.com/maps")
time.sleep(5)

# Search for software companies
search_box = driver.find_element(By.ID, "searchboxinput")
search_box.send_keys("software companies in Karachi")  # Change city as needed
search_box.send_keys(Keys.ENTER)
time.sleep(10)

# 🟡 Wait for results to appear
# Then find the scrollable sidebar using a safer method
try:
    scrollable = driver.find_element(By.XPATH, '//div[@role="feed"]')  # safer & stable
except:
    print("❌ Could not find scrollable sidebar.")
    driver.quit()
    exit()

# Scroll multiple times to load all results
for _ in range(20):  # Increase for more scrolling
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable)
    time.sleep(2)

# Get all unique place links
results = driver.find_elements(By.XPATH, '//a[contains(@href, "https://www.google.com/maps/place")]')
links = list(set([r.get_attribute("href") for r in results if r.get_attribute("href")]))
print(f"✅ Found {len(links)} places")

data = []

# Visit each place to get details
for link in links:
    try:
        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(5)

        try:
            name = driver.find_element(By.XPATH, '//h1').text
        except:
            name = "N/A"

        try:
            phone = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Phone:")]').get_attribute("aria-label").replace("Phone: ", "")
        except:
            phone = "N/A"

        try:
            website = driver.find_element(By.XPATH, '//a[contains(@aria-label, "Website")]').get_attribute("href")
        except:
            website = "N/A"

        data.append({"Name": name, "Phone": phone, "Website": website})
        print(f"✔️ Added: {name}")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)

    except Exception as e:
        print(f"❌ Skipped: {e}")
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        continue

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("software_houses.csv", index=False)
print("✅ Data saved to software_houses.csv")
driver.quit()


✅ Found 116 places
✔️ Added: Shispare Pvt. Ltd.
✔️ Added: Finosys (Private) Limited
✔️ Added: Webtrica | Web Design Karachi | Web Design Pakistan | Web Design Company Karachi | SEO | Mobile App Development
✔️ Added: IT World – Business IT Solutions & Software Development
✔️ Added: Brand Verse Solutions
✔️ Added: CyferTech Solutions - Website development Company / Software House Karachi
✔️ Added: Letscodify Solutions
✔️ Added: Aymeetech Software House
✔️ Added: KalSoft Pakistan
✔️ Added: Best Web Development & Digital Marketing Company In Karachi Pakistan - Siwtech SMC-Private Limited.
✔️ Added: Aish Tycoons (Web Development Company)
✔️ Added: Cognitive IT Solutions
✔️ Added: RIKSOF
✔️ Added: Zapdas Technologies
✔️ Added: TAG - The Alpha Group | E-Commerce | Software Company in Karachi.
✔️ Added: Soliters
✔️ Added: iCreativez Technologies
✔️ Added: Rat Technologies l Web Development Services l Software Company l Web Design & Development
✔️ Added: Bit Technologies - Website development C

In [4]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def start_scraping():
    keyword = entry.get()
    if not keyword.strip():
        messagebox.showerror("Input Error", "Please enter a search keyword.")
        return

    messagebox.showinfo("Scraping Started", f"Searching for: {keyword}")

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://www.google.com/maps")
    time.sleep(5)

    # Search
    search_box = driver.find_element(By.ID, "searchboxinput")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(10)

    # Scrollable area
    try:
        scrollable = driver.find_element(By.XPATH, '//div[@role="feed"]')
    except:
        messagebox.showerror("Error", "Could not find results.")
        driver.quit()
        return

    # Scroll to load more results
    for _ in range(20):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable)
        time.sleep(2)

    # Extract links
    results = driver.find_elements(By.XPATH, '//a[contains(@href, "https://www.google.com/maps/place")]')
    links = list(set([r.get_attribute("href") for r in results if r.get_attribute("href")]))
    print(f"Found {len(links)} places")

    data = []

    for link in links:
        try:
            driver.execute_script(f"window.open('{link}', '_blank');")
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(5)

            name = driver.find_element(By.XPATH, '//h1').text or "N/A"
            try:
                phone = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Phone:")]').get_attribute("aria-label").replace("Phone: ", "")
            except:
                phone = "N/A"

            try:
                website = driver.find_element(By.XPATH, '//a[contains(@aria-label, "Website")]').get_attribute("href")
            except:
                website = "N/A"

            data.append({"Name": name, "Phone": phone, "Website": website})
            print(f"Added: {name}")

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)
        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue

    driver.quit()

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv("software_houses.csv", index=False)
    messagebox.showinfo("Done", f"✅ {len(data)} businesses saved to software_houses.csv")

# GUI
root = tk.Tk()
root.title("Google Maps Scraper")
root.geometry("400x200")

label = tk.Label(root, text="Enter your search keyword:")
label.pack(pady=10)

entry = tk.Entry(root, width=40)
entry.pack(pady=5)

btn = tk.Button(root, text="Search & Extract", command=start_scraping)
btn.pack(pady=20)

root.mainloop()

In [5]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def start_scraping():
    keyword = entry.get()
    if not keyword.strip():
        messagebox.showerror("Input Error", "Please enter a search keyword.")
        return

    messagebox.showinfo("Scraping Started", f"Searching for: {keyword}")

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://www.google.com/maps")
    time.sleep(5)

    search_box = driver.find_element(By.ID, "searchboxinput")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(10)

    try:
        scrollable = driver.find_element(By.XPATH, '//div[@role="feed"]')
    except:
        messagebox.showerror("Error", "Could not find results.")
        driver.quit()
        return

    for _ in range(20):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable)
        time.sleep(2)

    results = driver.find_elements(By.XPATH, '//a[contains(@href, "https://www.google.com/maps/place")]')
    links = list(set([r.get_attribute("href") for r in results if r.get_attribute("href")]))
    print(f"Found {len(links)} places")

    data = []

    for link in links:
        try:
            driver.execute_script(f"window.open('{link}', '_blank');")
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(5)

            name = driver.find_element(By.XPATH, '//h1').text or "N/A"
            try:
                phone = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Phone:")]').get_attribute("aria-label").replace("Phone: ", "")
            except:
                phone = "N/A"

            try:
                website = driver.find_element(By.XPATH, '//a[contains(@aria-label, "Website")]').get_attribute("href")
            except:
                website = "N/A"

            data.append({"Name": name, "Phone": phone, "Website": website})
            print(f"Added: {name}")

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)
        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue

    driver.quit()

    df = pd.DataFrame(data)
    df.to_csv("software_houses.csv", index=False)
    messagebox.showinfo("Done", f"✅ {len(data)} businesses saved to software_houses.csv")

# ===================== GUI =====================
root = tk.Tk()
root.title("Google Maps Scraper")
root.geometry("600x300")  # 👈 Bigger window

# Background color
root.configure(bg="#f0f4f8")

# Title Label
title = tk.Label(root, text="🗺️ Google Maps Business Scraper", font=("Helvetica", 18, "bold"), bg="#f0f4f8", fg="#333")
title.pack(pady=20)

# Input Label
label = tk.Label(root, text="Enter your search keyword (e.g. software house in Lahore):", font=("Helvetica", 12), bg="#f0f4f8")
label.pack(pady=5)

# Entry box
entry = tk.Entry(root, width=50, font=("Helvetica", 12))
entry.pack(pady=5)

# Search Button
btn = tk.Button(root, text="Search & Extract", command=start_scraping, bg="#007acc", fg="white", font=("Helvetica", 12, "bold"), padx=10, pady=5)
btn.pack(pady=20)

# Run the GUI
root.mainloop()


Found 117 places
Added: Aen Ali Hashmi - SEO Expert Services & Digital Marketing Agency in Lahore, Pakistan
Added: BDM Solutions | SEO Services in Lahore | Digital Marketing Agency In Lahore
Added: Bee Buzz - Digital Marketing Agency in Lahore
Added: Digital Breeze - Digital Marketing Services in Lahore Pakistan
Added: Zain Ahsan | Google Ads & SEO Expert in Lahore | Rankers Nest
Added: SEO Services in Lahore | Digital Marketing Agency Pakistan | Digital PR
Added: Webfoxs | Best Digital Marketing and Web development Company In Lahore
Added: Digital Marketing Agency in Pakistan - I2m SEO
Added: FulFillit | Digital Marketing Agency Lahore
Added: ALM Advertising Agency
Added: Arslan Rauf - SEO Specialist | Digital Marketing Consultant GMB & Google adwords PPC expert in Lahore Pakistan
Added: digital marketing agency lahore
Added: Blue Digital pixel Digital marketing agency in Lahore Pakistan
Added: OYS Digital Marketing Agency
Added: Ranking Solutions | Digital Marketing Agency Lahore
Add